# Heart Rate Estimation using TAO HeartRateNet

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://d29g4g2dyqv443.cloudfront.net/sites/default/files/akamai/TAO/tlt-tao-toolkit-bring-your-own-model-diagram.png" width="1080"> 

## Learning Objectives
In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Take a pretrained model and train a model on the COHFACE dataset
* Run Inference on the trained model
* Export the retrained model to a .etlt file for deployment to DeepStream SDK

At the end of this notebook, you will have generated a trained and optimized `heartratenet` model, 
which you may deploy via [DeepStream](https://developer.nvidia.com/deepstream-sdk).

### Table of Contents

This notebook shows an example of non-invasive heart rate estimation using the Train Adapt Optimize (TAO) Toolkit.

0. [Set up env variables, map drives, and install dependencies](#head-0)
1. [Install the TAO launcher](#head-1)
2. [Prepare dataset and pre-trained model](#head-2) <br>
    2.1 [Verify downloaded dataset](#head-2-1) <br>
    2.2 [Extract video data to image data](#head-2-2) <br>
    2.2 [Process the extracted data](#head-2-3) <br>
    2.2 [Download pre-trained model](#head-2-4) <br>
3. [Generate tfrecords from RGB videos](#head-3) <br>
    3.1 [Download haarcascade classifier](#head-3-1) <br>
    3.2 [Generate tfrecords](#head-3-2) <br>
4. [Provide training specification](#head-4) <br>
5. [Run TAO training](#head-5) <br>
6. [Evaluate the trained model](#head-6) <br>
7. [Inference](#head-7) <br>
8. [Export](#head-8) <br>

## 0. Set up env variables, map drives and install dependencies <a class="anchor" id="head-0"></a>

When using the purpose-built pretrained models from NGC, please make sure to set the `$KEY` environment variable to the key as mentioned in the model overview. Failing to do so, can lead to errors when trying to load them as pretrained models.

The following notebook requires the user to set an env variable called the `$LOCAL_PROJECT_DIR` as the path to the users' workspace. Please note that the dataset to run this notebook is expected to reside in the `$LOCAL_PROJECT_DIR/heartratenet/data`, while the TAO experiment generated collaterals will be output to `$LOCAL_PROJECT_DIR/heartratenet`. More information on how to set up the dataset and the supported steps in the TAO workflow are provided in the subsequent cells.

*Note: This notebook currently is by default set up to run training using 1 GPU. To use more GPU's please update the env variable `$NUM_GPUS` accordingly*

In [ ]:
# Setting up env variables for cleaner command-line commands.
import os

%env KEY=nvidia_tlt
%env NUM_GPUS=1
%env USER_EXPERIMENT_DIR=/workspace/tao-experiments/heartratenet
%env HEARTRATENET_DATA=/workspace/tao-experiments/heartratenet/data

# Set this path if you don't run the notebook from the samples directory.
# %env NOTEBOOK_ROOT=~/tao-samples/heartratenet

# Please define this local project directory that needs to be mapped to the TAO docker session.
# The dataset is expected to be present in $LOCAL_PROJECT_DIR/heartratenet/data, while the results for the steps
# in this notebook will be stored at $LOCAL_PROJECT_DIR/heartratenet
# !PLEASE MAKE SURE TO UPDATE THIS PATH!.
%env LOCAL_PROJECT_DIR=/path/to/local/experiments

# $PROJECT_DIR is the path to the sample notebook folder and the dependency folder
# $PROJECT_DIR/deps should exist for dependency installation
%env PROJECT_DIR=/path/to/local/samples_dir

os.environ["LOCAL_DATA_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "heartratenet/data"
)
os.environ["LOCAL_EXPERIMENT_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "heartratenet"
)

# The sample spec files are present in the same path as the downloaded samples.
os.environ["LOCAL_SPECS_DIR"] = os.path.join(
    os.getenv("NOTEBOOK_ROOT", os.getcwd()),
    "specs"
)
%env SPECS_DIR=/workspace/tao-experiments/heartratenet/specs

# Showing list of specification files.
!ls -rlt $LOCAL_SPECS_DIR
!mkdir -p $LOCAL_EXPERIMENT_DIR/model/

The cell below maps the project directory on your local host to a workspace directory in the TAO docker instance, so that the data and the results are mapped from in and out of the docker. For more information please refer to the [launcher instance](https://docs.nvidia.com/tao/tao-toolkit/text/tao_launcher.html) in the user guide.

When running this cell on AWS, update the drive_map entry with the dictionary defined below, so that you don't have permission issues when writing data into folders created by the TAO docker.

```json
drive_map = {
    "Mounts": [
            # Mapping the data directory
            {
                "source": os.environ["LOCAL_PROJECT_DIR"],
                "destination": "/workspace/tao-experiments"
            },
            # Mapping the specs directory.
            {
                "source": os.environ["LOCAL_SPECS_DIR"],
                "destination": os.environ["SPECS_DIR"]
            },
        ],
    "DockerOptions": {
        "user": "{}:{}".format(os.getuid(), os.getgid())
    }
}
```

In [ ]:
# Mapping up the local directories to the TAO docker.
import json
mounts_file = os.path.expanduser("~/.tao_mounts.json")

# Define the dictionary with the mapped drives
drive_map = {
    "Mounts": [
        # Mapping the data directory
        {
            "source": os.environ["LOCAL_PROJECT_DIR"],
            "destination": "/workspace/tao-experiments"
        },
        # Mapping the specs directory.
        {
            "source": os.environ["LOCAL_SPECS_DIR"],
            "destination": os.environ["SPECS_DIR"]
        },
    ]
}

# Writing the mounts file.
with open(mounts_file, "w") as mfile:
    json.dump(drive_map, mfile, indent=4)

In [ ]:
!cat ~/.tao_mounts.json

In [ ]:
# Install requirement
!pip3 install Cython==0.29.36
!pip3 install -r $PROJECT_DIR/deps/requirements-pip.txt
!pip3 install pyyaml

The heartratenet api uses the `$DATAIO_SPEC` and `$TRAIN_SPEC` yaml files to set up directories. 

* `$DATAIO_SPEC` is `$LOCAL_SPECS_DIR/heartratenet_data_generation.yaml`
* `$TRAIN_SPEC` is `$LOCAL_SPECS_DIR/heartratenet_tlt_pretrain.yaml`

In [ ]:
# Set up dataio and train experiment spec path and check if they exist

os.environ["DATAIO_SPEC"] = os.path.join(os.environ["SPECS_DIR"], 'heartratenet_data_generation.yaml')
os.environ["TRAIN_SPEC"] = os.path.join(os.environ["SPECS_DIR"], 'heartratenet_tlt_pretrain.yaml')
os.environ["LOCAL_DATAIO_SPEC"] = os.path.join(os.environ["LOCAL_SPECS_DIR"], 'heartratenet_data_generation.yaml')
os.environ["LOCAL_TRAIN_SPEC"] = os.path.join(os.environ["LOCAL_SPECS_DIR"], 'heartratenet_tlt_pretrain.yaml')

!if [ ! -f $LOCAL_DATAIO_SPEC ]; then echo "Dataio spec file not found."; else echo "Found dataio spec file.";fi
!if [ ! -f $LOCAL_TRAIN_SPEC ]; then echo "Train spec file not found."; else echo "Found train spec file.";fi

Now, we have to make sure the environment paths selected above match the inputs to the api.
Go to `$DATAIO_SPEC` file and change `input_directory_path` and `data_directory_output_path` to the path specified in `$HEARTRATENET_DATA`.
Go to `$TRAIN_SPEC` file and change the `results_dir` to `$USER_EXPERIMENT_DIR` and also change the `checkpoint_dir` accordingly. This file is the input to the model. Next, change the `tfrecords_directory_path` to `$HEARTRATENET_DATA`. 

In [ ]:
# Check to see if spec files are found and is updated.
try:
    from yaml import load, SafeLoader
    import os
    from os.path import normpath
    
    with open(os.environ.get("LOCAL_DATAIO_SPEC")) as f:
        print('dataio spec found')
        dataio_args = load(f, Loader = SafeLoader)
    if normpath(dataio_args['input_directory_path'])!= normpath(os.environ.get("HEARTRATENET_DATA")) or normpath(dataio_args['data_directory_output_path']) != normpath(os.environ.get("HEARTRATENET_DATA")):
        print(normpath(dataio_args['input_directory_path']), os.environ.get("HEARTRATENET_DATA") )
        print('Please update input_directory_path and data_directory_output_path')
except:
    print('Dataio spec is not found, please ensure there is dataio spec in proper folder')
    
try:
    from yaml import load
    import os
    from os.path import normpath, join
    
    with open(os.environ.get("LOCAL_TRAIN_SPEC")) as f:
        print('train spec found')
        train_args = load(f, Loader=SafeLoader)
        
    if normpath(train_args['results_dir']) != normpath(os.environ.get("USER_EXPERIMENT_DIR")):
        print('Please update results_dir')
        
    if normpath(train_args['dataloader']['dataset_info']['tfrecords_directory_path'])!= normpath(os.environ.get("HEARTRATENET_DATA")):
        print('Please update the tfrecords_directory_path')
except:
    print('Train spec is not found, please ensure there is train spec in proper folder')

## 1. Install the TAO launcher <a class="anchor" id="head-1"></a>
The TAO launcher is a python package distributed as a python wheel listed in PyPI. You may install the launcher by executing the following cell.

Please note that TAO Toolkit recommends users run the TAO launcher in a virtual env with python 3.6.9. You may follow the instruction on this [page](https://virtualenvwrapper.readthedocs.io/en/latest/install.html) to set up a python virtual env using the `virtualenv` and `virtualenvwrapper` packages. Once you have set up virtualenvwrapper, please set the version of python to be used in the virtual env by using the `VIRTUALENVWRAPPER_PYTHON` variable. You may do so by running

```sh
export VIRTUALENVWRAPPER_PYTHON=/path/to/bin/python3.x
```
where x >= 6 and <= 8

We recommend performing this step first and then launching the notebook from the virtual environment. In addition to installing TAO python package, please make sure of the following software requirements:
* python >=3.7, <=3.10.x
* docker-ce > 19.03.5
* docker-API 1.40
* nvidia-container-toolkit > 1.3.0-1
* nvidia-container-runtime > 3.4.0-1
* nvidia-docker2 > 2.5.0-1
* nvidia-driver > 455+

Once you have installed the pre-requisites, please log in to the docker registry nvcr.io by following the command below

```sh
docker login nvcr.io
```

You will be triggered to enter a username and password. The username is `$oauthtoken` and the password is the API key generated from `ngc.nvidia.com`. Please follow the instructions in the [NGC setup guide](https://docs.nvidia.com/ngc/ngc-overview/index.html#generating-api-key) to generate your own API key.

In [ ]:
# SKIP this step IF you have already installed the TAO launcher wheel.
!pip3 install nvidia-tao

In [ ]:
# View the versions of the TAO launcher
!tao info

## 2. Prepare dataset and pre-trained model <a class="anchor" id="head-2"></a>

Please download COHFACE public dataset from the following website: https://www.idiap.ch/en/dataset/cohface

After downloading the data, please extract the data to cohface folder and place it under `$LOCAL_DATA_DIR`.

### A. Verify downloaded dataset <a class="anchor" id="head-2-1"></a>

In [ ]:
# Check the dataset is present.
!mkdir -p $LOCAL_DATA_DIR

In [ ]:
!if [ ! -d $LOCAL_DATA_DIR/cohface ]; then echo "Data folder not found, please download."; else echo "Data folder found.";fi

### B. Process the extracted data <a class="anchor" id="head-2-3"></a>

The `dataio` module for heartratenet expects the data to be formatted in a predefined format.

The `dataio` spec file specifies the folders to be read in the three lists train_subjects, validation_subjects and test_subjects.
Place the video file under each subject in a folder named images. The path is `$LOCAL_DATA_DIR/subject_folder`.

The ground truth is expected in the following format. For the RGB camera feed, `image_timestamps.csv` consists of frame ID and corresponding timestamp in rows `ID`,`Time`. For the pulse readings, `ground_truth.csv` consists of a timestamp and the corresponding ppg reading in rows `Time`,`PulseWaveform`. The heart rate is predicted as the dominant frequency of the ppg signal. The API takes care of sampling differences between the RGB and PPG signals. COHFACE dataset has 40 subjects, use `start_subject_id` and `end_subject_id` as input arguments to the following script to specify subjects to process, `process_cohface.py` process subjects in range `[start_subject_id, end_subject_id)`

The following block will process the COHFACE dataset into a format consistent with heartratenet api.

In [ ]:
!python process_cohface.py -i $LOCAL_DATA_DIR/cohface/ \
                           -o $LOCAL_DATA_DIR/cohface_processed \
                           -start_subject_id 1 \
                           -end_subject_id 41

### C. Download pre-trained model <a class="anchor" id="head-2-4"></a>

Please follow the instructions in the following to download and verify the pretrained model for heartratenet.

For HeartRateNet pretrained model please download model: `nvidia/tao/heartratenet:trainable_v2.0`.

After download the pre-trained model, please place the files in `$LOCAL_EXPERIMENT_DIR/pretrain_models`
You will then have the following path

* pretrained model in `$LOCAL_EXPERIMENT_DIR/pretrain_models/heartratenet_vtrainable_v2.0/model.tlt`


In [ ]:
# Installing NGC CLI on the local machine.
## Download and install
%env CLI=ngccli_cat_linux.zip
!mkdir -p $LOCAL_PROJECT_DIR/ngccli

# Remove any previously existing CLI installations
!rm -rf $LOCAL_PROJECT_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $LOCAL_PROJECT_DIR/ngccli
!unzip -u "$LOCAL_PROJECT_DIR/ngccli/$CLI" -d $LOCAL_PROJECT_DIR/ngccli/
!rm $LOCAL_PROJECT_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli/ngc-cli:{}".format(os.getenv("LOCAL_PROJECT_DIR", ""), os.getenv("PATH", ""))

In [ ]:
# List models available in the model registry.
!ngc registry model list nvidia/tao/heartratenet:*

In [ ]:
# Create the target destination to download the model.
!mkdir -p $LOCAL_EXPERIMENT_DIR/pretrain_models/

In [ ]:
# Download the pretrained model from NGC
!ngc registry model download-version nvidia/tao/heartratenet:trainable_v2.0 \
    --dest $LOCAL_EXPERIMENT_DIR/pretrain_models/

In [ ]:
!ls -rlt $LOCAL_EXPERIMENT_DIR/pretrain_models/heartratenet_vtrainable_v2.0

In [ ]:
# Check the dataset is present
!if [ ! -f $LOCAL_EXPERIMENT_DIR/pretrain_models/heartratenet_vtrainable_v2.0/model.tlt ]; then echo 'Pretrain model file not found, please download.'; else echo 'Found Pretrain model file.';fi

## 3. Generate tfrecords from RGB videos <a class="anchor" id="head-3"></a>
* Download haarcascade classifier and prepare directory
* Generate required motion and appearance maps for attention network.
* Create the tfrecords using the tao command

### A. Download haarcascade classifier <a class="anchor" id="head-3-1"></a>
Obtain the haarcascade classifer from (https://github.com/opencv/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml)

After downloading the haarcascade classifier, please place `haarcascade_frontalface_default.xml` in `$LOCAL_DATA_DIR`

You will have the following path

* haarcascade file in
`$LOCAL_DATA_DIR/haarcascade_frontalface_default.xml`

In [ ]:
!wget https://github.com/opencv/opencv/raw/master/data/haarcascades/haarcascade_frontalface_default.xml -P $LOCAL_DATA_DIR/

Note: Please make sure the file has been downloaded successfully, failure to do so will result in the rest of the notebook not being operational.

In [ ]:
# Check to see if haar classifier is present.
!if [ ! -f $LOCAL_DATA_DIR/haarcascade_frontalface_default.xml ]; then echo "Classifier not found, please ensure classifier is in proper file"; else echo "Classifier found.";fi

### B. Generate tfrecords <a class="anchor" id="head-3-2"></a>

In [ ]:
!tao model heartratenet dataset_convert --experiment_spec_file $DATAIO_SPEC

In [ ]:
# Check to see if tfrecords are present.
!if [ ! -f $LOCAL_DATA_DIR/train.tfrecord ]; then echo "Did not find training file, please ensure training record is generated."; else echo "Found training record";fi
!if [ ! -f $LOCAL_DATA_DIR/validation.tfrecord ]; then echo "Did not find validation file, please ensure validation record is generated."; else echo "Found validation record";fi
!if [ ! -f $LOCAL_DATA_DIR/test.tfrecord ]; then echo "Did not find test file, please ensure test record is generated."; else echo "Found test record";fi

## 4. Provide training specification <a class="anchor" id="head-4"></a>
* Tfrecords for the training dataset
    * In order to use the newly generated tfrecords for training, update the `tfrecords_directory_path` parameter of `dataset_info` section in the spec file at `$TRAIN_SPEC`
* Pre-trained model path
    * Update `checkpoint_dir` in the spec file `$TRAIN_SPEC`
* Other training (hyper-)parameters such as batch size, number of epochs, learning rate, etc.


In [ ]:
!cat $LOCAL_TRAIN_SPEC

## 5. Run TAO training <a class="anchor" id="head-5"></a>
* Provide the sample spec file and the output directory location for models

In [ ]:
!tao model heartratenet train -e $TRAIN_SPEC \
                        -k $KEY \
                        -r $USER_EXPERIMENT_DIR/model

## 6. Evaluate the trained model <a class="anchor" id="head-6"></a>

In [ ]:
!tao model heartratenet evaluate -e $TRAIN_SPEC \
                           -k $KEY \
                           -m $USER_EXPERIMENT_DIR/model/ \
                           -r $USER_EXPERIMENT_DIR/eval_results

## 7. Inference <a class="anchor" id="head-7"></a>
* Ensure you have the required data format as indicated in the model card
* Modify `m` to the full model path for evaluation
* Modify `subject_infer_dir` and `subject` below to align with your data
* Modify `results_dir` to your desired result directory
* Modify `fps` to match inference data fps, COHFACE dataset recorded in 20fps

In [ ]:
!tao model heartratenet inference -m $USER_EXPERIMENT_DIR/model/model.tlt \
                            --subject_infer_dir $HEARTRATENET_DATA \
                            --subject cohface_processed/1/0 \
                            --results_dir $USER_EXPERIMENT_DIR \
                            --fps 20 \
                            -k $KEY \
                            -c channels_first

In [ ]:
import os
import cv2
import IPython.display
import PIL.Image

subject_infer_dir = os.environ['LOCAL_DATA_DIR']
subject = 'Subject1'
display_freq = 30
results_file = os.path.join(os.environ['LOCAL_EXPERIMENT_DIR'], 'results.txt')
with open(results_file, 'r') as file:
    results = file.read()

subject_work_dir = os.path.join(subject_infer_dir, subject, 'images')
cap = cv2.VideoCapture(os.path.join(subject_work_dir, '%04d.bmp'))
while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        IPython.display.display(PIL.Image.fromarray(frame))
    else:
        break
print(results)

## 8. Export <a class="anchor" id="head-8"></a>
* Modify `m` to your model directory path
* Modify `out_file` to your desired full output path

In [ ]:
# Check the model for conversion is present.
!if [ ! -f $LOCAL_EXPERIMENT_DIR/model/model.tlt ]; then echo "Model file not found, please download."; else echo "Model file found.";fi

In [ ]:
!tao model heartratenet export -m $USER_EXPERIMENT_DIR/model/model.tlt \
                         -k $KEY \
                         --out_file $USER_EXPERIMENT_DIR/exported_model.tlt